# 로지스틱 회귀 분석

## 이진 고객 이탈

고객의 웹사이트 광고를 제공하는 마케팅 대행사에서 당사의 고객이탈율이 꽤 높다는 사실을 알게됐습니다.<br>
그 회사에서는 즉시 고객 관리자들을 할당했지만, 어떤 고객이 이탈 할 것인지 예측하는 기계학습 모델을 만들어서 가장 이탈확률이 높은 고객에게 우선적으로 고객 관리자를 배치할 수 있기를 원합니다.<br>
고객이 잠재적 이탈 고객인지 여부를 분류하는 분류모델을 만드세요.

데이터는 customer_churn.csv로 저장돼있습니다. 아래는 각 컬럼의 정의입니다.
- Name : 회사의 최근 담당자의 이름
- Age : 고객의 나이
- Total_Purchase : 구매한 총 광고
- Account_Manager : 바이너리 0 = 고객 관리자 없음, 1 = 고객 관리자 할당됨
- Years : 거래 유지 연도수
- Num_sites : 서비스를 이용 중인 웹사이트 수
- Onboard_date : 마지막 연락처가 등록된 날짜
- Location : 고객 사무실 주소
- Company : 고객 회사의 이름

모델 생성 후에는 고객이 제공하는 new_customers.csv 파일에 저장된 새로운 데이터에 대한 예측값을 제시하세요. <br>
고객은 이 데이터를 통해 관리가 필요한 고객들을 알고 싶어 합니다.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

### 데이터 불러오기

In [2]:
df = pd.read_csv("./data/customer_churn.csv")

### 데이터 확인

In [3]:
df.head()

,Names,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Onboard_date,Location,Company,Churn
0,Cameron Williams,42.0,11066.80,0,7.22,8.0,2013-08-30 07:00:40,"10265 Elizabeth Mission Barkerburgh, AK 89518",Harvey LLC,1
1,Kevin Mueller,41.0,11916.22,0,6.50,11.0,2013-08-13 00:38:46,"6157 Frank Gardens Suite 019 Carloshaven, RI 1...",Wilson PLC,1
2,Eric Lozano,38.0,12884.75,0,6.67,12.0,2016-06-29 06:20:07,"1331 Keith Court Alyssahaven, DE 90114","Miller, Johnson and Wallace",1
3,Phillip White,42.0,8010.76,0,6.71,10.0,2014-04-22 12:43:12,"13120 Daniel Mount Angelabury, WY 30645-4695",Smith Inc,1
4,Cynthia Norton,37.0,9191.58,0,5.56,9.0,2016-01-19 15:31:15,"765 Tricia Row Karenshire, MH 71730",Love-Jones,1


In [4]:
df.shape

(900, 10)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Names            900 non-null    object 
 1   Age              900 non-null    float64
 2   Total_Purchase   900 non-null    float64
 3   Account_Manager  900 non-null    int64  
 4   Years            900 non-null    float64
 5   Num_Sites        900 non-null    float64
 6   Onboard_date     900 non-null    object 
 7   Location         900 non-null    object 
 8   Company          900 non-null    object 
 9   Churn            900 non-null    int64  
dtypes: float64(4), int64(2), object(4)
memory usage: 70.4+ KB


- 결측치는 없음

In [6]:
df.describe()

,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Churn
count,900.000000,900.000000,900.000000,900.000000,900.000000,900.000000
mean,41.816667,10062.824033,0.481111,5.273156,8.587778,0.166667
std,6.127560,2408.644532,0.499921,1.274449,1.764836,0.372885
min,22.000000,100.000000,0.000000,1.000000,3.000000,0.000000
25%,38.000000,8497.122500,0.000000,4.450000,7.000000,0.000000
50%,42.000000,10045.870000,0.000000,5.215000,8.000000,0.000000
75%,46.000000,11760.105000,1.000000,6.110000,10.000000,0.000000
max,65.000000,18026.010000,1.000000,9.150000,14.000000,1.000000


### 머신러닝용 데이터 전처리

우리는 수치형 데이터만 이용할 예정입니다. Account_Manager는 다루기 어렵지 않기 때문에 모델학습에 포함할 수도 있겠지만 무작위로 할당된 것이기 때문에 큰 의미가 없을 수 있습니다.

- Onboard_date의 경우 날짜이지만, object 형식으로 지정되어 있음. 날짜 데이터에서 시간 단위는 크게 중요하지 않을 것으로 판단, 시간 단위는 삭제하고날짜 부분만 날짜 단위로 치환하여 데이터 분석에 사용
    - 날짜로 치환하더라도 모델 생성에 의미가 없을 가능성이 있으며, 그러할 경우 모델에서 제외
    - datetime64 타입의 경우 스케일러 적용이 불가능하므로, 해당 열 내의 가장 최근 날짜에서 각 날짜를 빼는 식으로 데이터 처리를 진행
    - 날짜 부분과 시간 부분은 띄어쓰기로 나뉘어 있는 것으로 보임

In [7]:
df["Onboard_date"].str.split(" ") # Onboard_date를 띄어쓰기 기준으로 나눔

0      [2013-08-30, 07:00:40]
1      [2013-08-13, 00:38:46]
2      [2016-06-29, 06:20:07]
3      [2014-04-22, 12:43:12]
4      [2016-01-19, 15:31:15]
                ...          
895    [2007-12-01, 13:29:34]
896    [2008-12-28, 15:23:58]
897    [2014-06-20, 05:10:09]
898    [2012-05-30, 00:15:43]
899    [2010-09-25, 12:16:08]
Name: Onboard_date, Length: 900, dtype: object

In [8]:
date = df["Onboard_date"].str.split(" ") # 나눈 날짜 데이터를 변수에 지정

In [9]:
df["Onboard_date_real"] = date.str.get(0) # 날짜에 해당되는 각 리스트의 0번째 값만 데이터프레임에 추가

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Names              900 non-null    object 
 1   Age                900 non-null    float64
 2   Total_Purchase     900 non-null    float64
 3   Account_Manager    900 non-null    int64  
 4   Years              900 non-null    float64
 5   Num_Sites          900 non-null    float64
 6   Onboard_date       900 non-null    object 
 7   Location           900 non-null    object 
 8   Company            900 non-null    object 
 9   Churn              900 non-null    int64  
 10  Onboard_date_real  900 non-null    object 
dtypes: float64(4), int64(2), object(5)
memory usage: 77.5+ KB


- 담긴 날짜는 object 형식으로 날짜 형식으로 변환

In [11]:
df["Onboard_date_real"] = pd.to_datetime(df["Onboard_date_real"])

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Names              900 non-null    object        
 1   Age                900 non-null    float64       
 2   Total_Purchase     900 non-null    float64       
 3   Account_Manager    900 non-null    int64         
 4   Years              900 non-null    float64       
 5   Num_Sites          900 non-null    float64       
 6   Onboard_date       900 non-null    object        
 7   Location           900 non-null    object        
 8   Company            900 non-null    object        
 9   Churn              900 non-null    int64         
 10  Onboard_date_real  900 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(2), object(4)
memory usage: 77.5+ KB


In [13]:
df.drop(["Onboard_date"], axis = 1, inplace = True) # Onboard_date 열은 삭제

In [14]:
df.head()

,Names,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Location,Company,Churn,Onboard_date_real
0,Cameron Williams,42.0,11066.80,0,7.22,8.0,"10265 Elizabeth Mission Barkerburgh, AK 89518",Harvey LLC,1,2013-08-30
1,Kevin Mueller,41.0,11916.22,0,6.50,11.0,"6157 Frank Gardens Suite 019 Carloshaven, RI 1...",Wilson PLC,1,2013-08-13
2,Eric Lozano,38.0,12884.75,0,6.67,12.0,"1331 Keith Court Alyssahaven, DE 90114","Miller, Johnson and Wallace",1,2016-06-29
3,Phillip White,42.0,8010.76,0,6.71,10.0,"13120 Daniel Mount Angelabury, WY 30645-4695",Smith Inc,1,2014-04-22
4,Cynthia Norton,37.0,9191.58,0,5.56,9.0,"765 Tricia Row Karenshire, MH 71730",Love-Jones,1,2016-01-19


In [15]:
df.drop(["Names", "Location", "Company"], axis = 1, inplace = True) # 수치형 데이터만 사용하기로 했으므로 문자 형식의 데이터는 모두 삭제

In [16]:
df.head()

,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Churn,Onboard_date_real
0,42.0,11066.80,0,7.22,8.0,1,2013-08-30
1,41.0,11916.22,0,6.50,11.0,1,2013-08-13
2,38.0,12884.75,0,6.67,12.0,1,2016-06-29
3,42.0,8010.76,0,6.71,10.0,1,2014-04-22
4,37.0,9191.58,0,5.56,9.0,1,2016-01-19


In [17]:
df["Onboard_date_real"].max()

Timestamp('2016-12-28 00:00:00')

In [18]:
D = df["Onboard_date_real"].max() # new_customers.csv 에 적용하기 위하여 최근날짜는 변수로 저장

In [19]:
df["Date_num"] = (D - df["Onboard_date_real"]).dt.days

In [20]:
df.head()

,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Churn,Onboard_date_real,Date_num
0,42.0,11066.80,0,7.22,8.0,1,2013-08-30,1216
1,41.0,11916.22,0,6.50,11.0,1,2013-08-13,1233
2,38.0,12884.75,0,6.67,12.0,1,2016-06-29,182
3,42.0,8010.76,0,6.71,10.0,1,2014-04-22,981
4,37.0,9191.58,0,5.56,9.0,1,2016-01-19,344


In [21]:
df.drop("Onboard_date_real", axis = 1, inplace = True) # 날짜 데이터 제거

In [22]:
df.head()

,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Churn,Date_num
0,42.0,11066.80,0,7.22,8.0,1,1216
1,41.0,11916.22,0,6.50,11.0,1,1233
2,38.0,12884.75,0,6.67,12.0,1,182
3,42.0,8010.76,0,6.71,10.0,1,981
4,37.0,9191.58,0,5.56,9.0,1,344


In [23]:
df.describe()

,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Churn,Date_num
count,900.000000,900.000000,900.000000,900.000000,900.000000,900.000000,900.000000
mean,41.816667,10062.824033,0.481111,5.273156,8.587778,0.166667,2087.830000
std,6.127560,2408.644532,0.499921,1.274449,1.764836,0.372885,1171.881625
min,22.000000,100.000000,0.000000,1.000000,3.000000,0.000000,0.000000
25%,38.000000,8497.122500,0.000000,4.450000,7.000000,0.000000,1047.500000
50%,42.000000,10045.870000,0.000000,5.215000,8.000000,0.000000,2121.000000
75%,46.000000,11760.105000,1.000000,6.110000,10.000000,0.000000,3134.500000
max,65.000000,18026.010000,1.000000,9.150000,14.000000,1.000000,4013.000000


In [24]:
X = df.drop(["Churn"], axis = 1) # 독립변수 / 종속변수 나누기
Y = df["Churn"]

In [25]:
X.head()

,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Date_num
0,42.0,11066.80,0,7.22,8.0,1216
1,41.0,11916.22,0,6.50,11.0,1233
2,38.0,12884.75,0,6.67,12.0,182
3,42.0,8010.76,0,6.71,10.0,981
4,37.0,9191.58,0,5.56,9.0,344


In [26]:
# 이탈율의 비율과 동일한 비율로 데이터를 나눔
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, stratify = Y, random_state = 1)

In [27]:
ss = StandardScaler() # 스케일러 적용
scaled_train = ss.fit_transform(x_train)
scaled_test = ss.transform(x_test)

In [28]:
logi = LogisticRegression() # 로지스틱 회귀 모델 생성 및 훈련
logi.fit(scaled_train, y_train)

LogisticRegression()

In [29]:
logi.coef_, logi.intercept_ # 회귀식 확인

(array([[0.28299831, 0.07815163, 0.13100431, 0.64481774, 1.93096422,
         0.01075432]]),
 array([-2.64476752]))

In [30]:
print(logi.score(scaled_train, y_train)) # 모델 평가
print(logi.score(scaled_test, y_test))

0.8857142857142857
0.9185185185185185


In [31]:
pred = logi.predict(scaled_test) # 모델 평가
ras = roc_auc_score(y_test, pred)
ras

0.7911111111111111

# 스케일러 적용 유무에 따른 점수 변화 확인

In [32]:
# 스케일러 미적용 모델 생성

X = df.drop(["Churn"], axis = 1) # 독립변수 / 종속변수 나누기
Y = df["Churn"]

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, stratify = Y, random_state = 1)

logi = LogisticRegression() # 로지스틱 회귀 모델 생성 및 훈련
logi.fit(x_train, y_train)

LogisticRegression()

In [33]:
logi.coef_, logi.intercept_ # 회귀식 확인

(array([[-1.14016468e-01, -2.03570610e-04,  4.80445116e-03,
          1.02627312e-01,  5.25901481e-01, -9.51168018e-05]]),
 array([-0.01589673]))

In [34]:
print(logi.score(x_train, y_train)) # 모델 평가
print(logi.score(x_test, y_test))

0.8380952380952381
0.8629629629629629


In [35]:
pred = logi.predict(x_test) # 모델 평가
ras = roc_auc_score(y_test, pred)
ras

0.6333333333333333

- 스케일러를 사용하지 않으면 성능이 대폭 하락하므로 스케일러는 사용

# Account_Manager 사용 여부에 따른 점수 변화 확인

In [36]:
df2 = df.drop("Account_Manager", axis = 1)

In [37]:
df2.head()

,Age,Total_Purchase,Years,Num_Sites,Churn,Date_num
0,42.0,11066.80,7.22,8.0,1,1216
1,41.0,11916.22,6.50,11.0,1,1233
2,38.0,12884.75,6.67,12.0,1,182
3,42.0,8010.76,6.71,10.0,1,981
4,37.0,9191.58,5.56,9.0,1,344


In [38]:
X = df2.drop(["Churn"], axis = 1) # 독립변수 / 종속변수 나누기
Y = df2["Churn"]

# 이탈율의 비율과 동일한 비율로 훈련 데이터와 테스트 데이터를 나눔
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, stratify = Y, random_state = 1)

ss = StandardScaler() # 스케일러 적용
scaled_train = ss.fit_transform(x_train)
scaled_test = ss.transform(x_test)

logi = LogisticRegression() # 로지스틱 회귀 모델 생성 및 훈련
logi.fit(scaled_train, y_train)

LogisticRegression()

In [39]:
logi.coef_, logi.intercept_ # 회귀식 확인

(array([[0.2793616 , 0.07793509, 0.64598885, 1.91770415, 0.00943855]]),
 array([-2.63252016]))

In [40]:
print(logi.score(scaled_train, y_train)) # 모델 평가
print(logi.score(scaled_test, y_test))

0.8857142857142857
0.9111111111111111


In [41]:
pred = logi.predict(scaled_test) # 모델 평가
ras = roc_auc_score(y_test, pred)
ras

0.768888888888889

- Account_Manager를 사용하는쪽의 성능이 좋으므로 모델에 사용

# Date_num 사용 여부에 따른 점수 변화 확인

In [42]:
df3 = df.drop("Date_num", axis = 1)

In [43]:
df3.head()

,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Churn
0,42.0,11066.80,0,7.22,8.0,1
1,41.0,11916.22,0,6.50,11.0,1
2,38.0,12884.75,0,6.67,12.0,1
3,42.0,8010.76,0,6.71,10.0,1
4,37.0,9191.58,0,5.56,9.0,1


In [44]:
X = df3.drop(["Churn"], axis = 1) # 독립변수 / 종속변수 나누기
Y = df3["Churn"]

# 이탈율의 비율과 동일한 비율로 훈련 데이터와 테스트 데이터를 나눔
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, stratify = Y, random_state = 1)

ss = StandardScaler() # 스케일러 적용
scaled_train = ss.fit_transform(x_train)
scaled_test = ss.transform(x_test)

logi = LogisticRegression() # 로지스틱 회귀 모델 생성 및 훈련
logi.fit(scaled_train, y_train)

LogisticRegression()

In [45]:
logi.coef_, logi.intercept_ # 회귀식 확인

(array([[0.28340367, 0.07846995, 0.13090021, 0.64561472, 1.93158358]]),
 array([-2.64509259]))

In [46]:
print(logi.score(scaled_train, y_train)) # 모델 평가
print(logi.score(scaled_test, y_test))

0.8857142857142857
0.9185185185185185


In [47]:
pred = logi.predict(scaled_test) # 모델 평가
ras = roc_auc_score(y_test, pred)
ras

0.7911111111111111

- Date_num (= Onboard_date)을 사용하지 않아도 점수 변화는 거의 없으므로 사용하지 않음
    - 최초 모델 생성에서도 기울기 값이 매우 작게 나왔음

# 변수 가공의 가능성

- Total_Purchase에서 Years를 나누면 연간 평균 광고 구매수가 나오므로 새로운 독립변수를 생성하여 모델 생성에 활용해볼수 있음

In [48]:
df_alt = df[:] # 데이터프레임은 별도 주소로 저장하여 사용

In [49]:
df_alt.head()

,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Churn,Date_num
0,42.0,11066.80,0,7.22,8.0,1,1216
1,41.0,11916.22,0,6.50,11.0,1,1233
2,38.0,12884.75,0,6.67,12.0,1,182
3,42.0,8010.76,0,6.71,10.0,1,981
4,37.0,9191.58,0,5.56,9.0,1,344


In [50]:
df_alt["Total_Purchase"] / df_alt["Years"]

0      1532.797784
1      1833.264615
2      1931.746627
3      1193.853949
4      1653.161871
          ...     
895    3536.138122
896    1431.826339
897    2208.091575
898    1191.577697
899    1855.697211
Length: 900, dtype: float64

In [51]:
df_alt["Purchase_per_year"] = df_alt["Total_Purchase"] / df_alt["Years"]

C:\Users\yjg10\AppData\Local\Temp\ipykernel_8584\1860965848.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_alt["Purchase_per_year"] = df_alt["Total_Purchase"] / df_alt["Years"]


In [52]:
df_alt.head()

,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Churn,Date_num,Purchase_per_year
0,42.0,11066.80,0,7.22,8.0,1,1216,1532.797784
1,41.0,11916.22,0,6.50,11.0,1,1233,1833.264615
2,38.0,12884.75,0,6.67,12.0,1,182,1931.746627
3,42.0,8010.76,0,6.71,10.0,1,981,1193.853949
4,37.0,9191.58,0,5.56,9.0,1,344,1653.161871


In [53]:
# Date_num은 삭제
df_alt.drop(["Date_num"], axis = 1, inplace = True)

In [54]:
df_alt.head()

,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Churn,Purchase_per_year
0,42.0,11066.80,0,7.22,8.0,1,1532.797784
1,41.0,11916.22,0,6.50,11.0,1,1833.264615
2,38.0,12884.75,0,6.67,12.0,1,1931.746627
3,42.0,8010.76,0,6.71,10.0,1,1193.853949
4,37.0,9191.58,0,5.56,9.0,1,1653.161871


In [55]:
X = df_alt.drop(["Churn"], axis = 1) # 독립변수 / 종속변수 나누기
Y = df_alt["Churn"]

In [56]:
X.head()

,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Purchase_per_year
0,42.0,11066.80,0,7.22,8.0,1532.797784
1,41.0,11916.22,0,6.50,11.0,1833.264615
2,38.0,12884.75,0,6.67,12.0,1931.746627
3,42.0,8010.76,0,6.71,10.0,1193.853949
4,37.0,9191.58,0,5.56,9.0,1653.161871


In [57]:
# 이탈율의 비율과 동일한 비율로 훈련 데이터와 테스트 데이터를 나눔
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, stratify = Y, random_state = 1)

In [58]:
ss = StandardScaler() # 스케일러 적용
scaled_train = ss.fit_transform(x_train)
scaled_test = ss.transform(x_test)

In [59]:
logi = LogisticRegression() # 로지스틱 회귀 모델 생성 및 훈련
logi.fit(scaled_train, y_train)

LogisticRegression()

In [60]:
logi.coef_, logi.intercept_ # 회귀식 확인

(array([[ 0.2853926 ,  0.18314278,  0.13148522,  0.54830492,  1.9313488 ,
         -0.17455362]]),
 array([-2.64931641]))

In [61]:
print(logi.score(scaled_train, y_train)) # 모델 평가
print(logi.score(scaled_test, y_test))

0.8873015873015873
0.9148148148148149


In [62]:
pred = logi.predict(scaled_test) # 모델 평가
ras = roc_auc_score(y_test, pred)
ras

0.788888888888889

- 해당 독립변수 가공은 사용하지 않는 쪽이 성능에 좋음

# 결론

- 최종적으로는 Date_num 미사용, 스케일러 사용, Account_Manager을 사용하는쪽이 모델 성능이 가장 좋게 나왔음

In [63]:
df3 = df.drop("Date_num", axis = 1)

In [64]:
df3.head()

,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Churn
0,42.0,11066.80,0,7.22,8.0,1
1,41.0,11916.22,0,6.50,11.0,1
2,38.0,12884.75,0,6.67,12.0,1
3,42.0,8010.76,0,6.71,10.0,1
4,37.0,9191.58,0,5.56,9.0,1


In [65]:
X = df3.drop(["Churn"], axis = 1) # 독립변수 / 종속변수 나누기
Y = df3["Churn"]

# 이탈율의 비율과 동일한 비율로 훈련 데이터와 테스트 데이터를 나눔
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, stratify = Y, random_state = 1)

ss = StandardScaler() # 스케일러 적용
scaled_train = ss.fit_transform(x_train)
scaled_test = ss.transform(x_test)

logi = LogisticRegression() # 로지스틱 회귀 모델 생성 및 훈련
logi.fit(scaled_train, y_train)

LogisticRegression()

In [66]:
logi.coef_, logi.intercept_ # 회귀식 확인

(array([[0.28340367, 0.07846995, 0.13090021, 0.64561472, 1.93158358]]),
 array([-2.64509259]))

In [67]:
print(logi.score(scaled_train, y_train)) # 모델 평가
print(logi.score(scaled_test, y_test))

0.8857142857142857
0.9185185185185185


In [68]:
pred = logi.predict(scaled_test) # 모델 평가
ras = roc_auc_score(y_test, pred)
ras

0.7911111111111111

# 실제 제공 데이터를 이용한 예측

In [69]:
real_df = pd.read_csv("./data/new_customers.csv", index_col = False)
# 그냥 불러오면 이유는 알 수 없으나 Names 열이 인덱스 넘버가 됨. 따라서 index_col을 False로 지정하여 해당 현상을 방지

In [70]:
real_df.head()

,Names,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Onboard_date,Location,Company
0,Andrew Mccall,37.0,9935.53,1,7.71,8.0,2011-08-29 18:37:54,"38612 Johnny Stravenue Nataliebury, WI 15717-8316",King Ltd
1,Michele Wright,23.0,7526.94,1,9.28,15.0,2013-07-22 18:19:54,"21083 Nicole Junction Suite 332, Youngport, ME...",Cannon-Benson
2,Jeremy Chang,65.0,100.00,1,1.00,15.0,2006-12-11 07:48:13,"085 Austin Views Lake Julialand, WY 63726-4298",Barron-Robertson
3,Megan Ferguson,32.0,6487.50,0,9.40,14.0,2016-10-28 05:32:13,"922 Wright Branch North Cynthialand, NC 64721",Sexton-Golden
4,Taylor Young,32.0,13147.71,1,10.00,8.0,2012-03-20 00:36:46,Unit 0789 Box 0734 DPO AP 39702,Wood LLC


In [71]:
real_df.drop(["Names", "Onboard_date", "Location", "Company"], axis = 1, inplace = True)

In [72]:
real_df.head()

,Age,Total_Purchase,Account_Manager,Years,Num_Sites
0,37.0,9935.53,1,7.71,8.0
1,23.0,7526.94,1,9.28,15.0
2,65.0,100.00,1,1.00,15.0
3,32.0,6487.50,0,9.40,14.0
4,32.0,13147.71,1,10.00,8.0


In [73]:
scaled_real = ss.transform(real_df)

In [74]:
pred = logi.predict(scaled_real)

In [75]:
pred

array([0, 1, 1, 1, 0, 1], dtype=int64)